# Student ID X20108851

In [23]:
import findspark
import pyspark
#from pyspark import SparkContext
from pyspark import SparkContext
from pyspark.sql import SparkSession

import seaborn as sns
import re
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, ImageColorGenerator
from textblob import TextBlob

In [24]:
import pyspark
import time
from pyspark.sql import Row

# Create Spark Session

In [25]:
#SparkSession.stop(spark)
spark=SparkSession.builder.master("local").appName("first").getOrCreate()

In [26]:
sc = spark.sparkContext
nums = sc.parallelize([1, 2, 3, 4])
squared = nums.map(lambda x: x * x).collect()
for num in squared:
    print("%i " % (num))

1 
4 
9 
16 


# Load Data

In [27]:

time1 = time.time()

data=sc.textFile("/user/hduser/projectData/DIAdata/FlumeData_*")

time2 = time.time()
print(time2 - time1, ' Seconds')

0.3609459400177002  Seconds


# Tweets data pre-processing

In [28]:
time3 = time.time()

In [29]:
#Split the Data
data_split=data.flatMap(lambda x:x.split(" "))

In [30]:
#Filter only the hashtags from the data.
hashtags=data_split.filter(lambda x:"#" in x)

#Assign a tuple value of key-value pair(x,1) to all elements to calculate number of similar tweets.
tweet=hashtags.map(lambda x:(x,1))

In [31]:
#Since RDD has a tuple of key-value pair =(tweet,1) , add all the values with similar keys.
add=tweet.reduceByKey(lambda x,y:x+y)

#set the count in the decreasing order to retrieve only the most popular tweets.
list=add.map(lambda x:(x[1],x[0]))


In [32]:
time4 = time.time()
print(time4 - time3, ' Seconds')

0.6165299415588379  Seconds


In [33]:
desc=list.sortByKey(ascending=False)

#Now the RDD has a key-value pair of key as tweet with value of total counts.
#create a Dataframe to run queries
df=desc.map(lambda x:Row(tweet_count=x[0],tweet=x[1]))


In [34]:
df=df.toDF()

In [35]:
df.head()

Row(tweet_count=1880, tweet='#Travala')

# SQL Queries

In [36]:
#register the dataframe as a View table to run SQL queries
df.createOrReplaceTempView('table')

#now query the data using sparkSQL
query=spark.sql("SELECT * FROM table limit 15")


In [37]:
query.show()

+-----------+--------------------+
|tweet_count|               tweet|
+-----------+--------------------+
|       1880|            #Travala|
|       1760|             #travel|
|       1760|         #FrizeMedia|
|       1440|href=\"http://twi...|
|       1320|           \n\n#DOGE|
|       1220|#TeacherAppreciat...|
|       1160|#MizzouMade","cre...|
|        980|              #wileg|
|        980|             #Travel|
|        640|            #tourism|
|        580|             #9Today|
|        580|#9Today","created...|
|        540|#BTC","created_at...|
|        540|  #travelphotography|
|        540|                #BNB|
+-----------+--------------------+



In [38]:
time5 = time.time()
print(time5 - time4, ' Seconds')

13.192201375961304  Seconds


now there might be few incorrect hastags in the table , so retreive only the proper tweets

In [39]:

result=spark.sql("SELECT * FROM table WHERE table.tweet LIKE '#%'")



In [40]:
#display the results
result.show(50) 

+-----------+--------------------+
|tweet_count|               tweet|
+-----------+--------------------+
|       1880|            #Travala|
|       1760|             #travel|
|       1760|         #FrizeMedia|
|       1220|#TeacherAppreciat...|
|       1160|#MizzouMade","cre...|
|        980|              #wileg|
|        980|             #Travel|
|        640|            #tourism|
|        580|             #9Today|
|        580|#9Today","created...|
|        540|#BTC","created_at...|
|        540|  #travelphotography|
|        540|                #BNB|
|        460|      #travelblogger|
|        440|                #SEO|
|        440|            #Content|
|        440|        #FrizeMedia.|
|        440|        #Advertising|
|        440|            #Founder|
|        440|     #ThoughtLeader,|
|        440|#InfluencerMarketing|
|        440|      #GrowthHacking|
|        440|#BrandAdvocacy","...|
|        440|            #Digital|
|        440|         #Marketing.|
|        440|  #Digi